# TP01: Archivos netCDF Image


In [ ]:
import datetime as dt
import os
from numbers import Number

import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr

# librerias graficas
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def kelvin2celcius(t):
    return t -273.15

def timezero(unit, unit_format):
    return dt.datetime.strptime(unit, unit_format)

def floatdays_to_timedelta(d):
    days = int(np.trunc(d))
    h = 24 * (d - days)
    hours = int(h)
    m = 60 * (h - hours)
    minutes = int(m)
    s = 60 * (m - minutes)
    seconds = int(s)
    td = dt.timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
    return td

def copernicus_to_datetime(time, unit=None, unit_format=None):
    time_zero = timezero(unit, unit_format)
    if isinstance(time, Number):
        return time_zero + floatdays_to_timedelta(time)
    else:
        times = [time_zero + floatdays_to_timedelta(t) for t in time]
        return np.asarray(times)
    
def date_to_str(t, tformat):
    return dt.datetime.strftime(t, tformat)

def to_ticks(time_zero, delta, n):
    return [time_zero + i * delta for i in range(n)]

def outofrange_to_nan(matrix, minvalue=0, maxvalue=100):
    return np.where(np.logical_and(matrix>minvalue, matrix<maxvalue), matrix, np.nan)


DATADIR = "./data"

## ARCHIVO: METOFFICE-GLO-SST-L4-RAN-OBS-SST_1574878238997

In [ ]:
nf = "METOFFICE-GLO-SST-L4-RAN-OBS-SST_1574878238997.nc" 
ds = nc.Dataset(os.path.join(DATADIR, nf))

### **Visualice los atributos del archivo (origen de los datos, fecha de creación, unidades, etc)**



In [ ]:
for attr in ds.ncattrs():
    print(f"{attr}: ", ds.getncattr(attr))

**variables y dimensiones**



In [ ]:
for key, var in ds.variables.items():
    print(f"{key}: ")
    for attr in var.ncattrs():
        print(f"\t{attr}: {var.getncattr(attr)}")

In [ ]:
for key, dim in ds.dimensions.items():
    print(f"dimension: {dim.name}, size: {dim.size}")

In [ ]:
lon = ds.variables["lon"][:]
lat = ds.variables["lat"][:]
time = ds.variables["time"][:]
sst = ds.variables["analysed_sst"][:]
err = ds.variables["analysis_error"][:]
sst = kelvin2celcius(sst)

### **a. Mapa lat-lon con diferentes proyecciones.**


Link con todas las  projecciones disponibles:  
https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html 



#### Projeccion PlateCarree

In [ ]:
fig = plt.figure(figsize=(8, 6))
fig.subplots_adjust(top=10, bottom=9.5)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

CS = ax.contourf(lon, lat, sst[0, :, :], 10, cmap='bwr', zorder=0)
cbar = fig.colorbar(CS, 
                   orientation='vertical',
                   label='Temperatura (°C)',
                   shrink=0.7)

# agrego el contorno entre valores y su valor
cs = plt.contour(lon, lat, sst[0, :, :], 5, colors='k', linestyles="dashed", alpha=1)
plt.clabel(cs, inline=1, fmt='%1.1f')

plt.title('Temperatura superficial del Mar', loc='center', fontsize=15)
g = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,  zorder=0)
g.ylabels_right = False
g.xlabels_top = False
g.yformatter = LATITUDE_FORMATTER
g.xformatter = LONGITUDE_FORMATTER
g.xlabel_style={'size':12}
g.ylabel_style={'size':12}

#### Projeccion Orthographic

In [ ]:
# Projeccion Orthograaphic
fig = plt.figure(figsize=(8, 6))
fig.subplots_adjust(top=10, bottom=9.5)
ax = fig.add_subplot(111, projection=ccrs.Orthographic(-60,-30))


ax.add_feature(cfeature.LAND, facecolor='white', zorder=1)
# ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)


CS = ax.contourf(lon, lat, sst[0, :], 10, cmap='bwr', transform=ccrs.PlateCarree(), zorder=0)
cbar = fig.colorbar(CS, orientation='vertical', label='Temperatura (°C)', shrink=0.7)

# agrego el contorno entre valores y su valor
cs = plt.contour(lon, lat, sst[0, :, :], 5, colors='k', linestyles="dashed", alpha=1, transform=ccrs.PlateCarree())
plt.clabel(cs, inline=1, fmt='%1.1f')

ax.add_feature(cfeature.LAND,facecolor='silver')
ax.add_feature(cfeature.BORDERS, linestyle=':')

plt.title('Temperatura superficial del Mar', loc='center', fontsize=15)
g = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,  zorder=0)
g.ylabels_right = False
g.xlabels_top = False
g.yformatter = LATITUDE_FORMATTER
g.xformatter = LONGITUDE_FORMATTER
g.xlabel_style={'size':12}
g.ylabel_style={'size':12}

#### Projeccion SouthPolarStereo

In [ ]:
fig = plt.figure(figsize=(8, 6))
fig.subplots_adjust(top=10, bottom=9.5)
ax = fig.add_subplot(111, projection=ccrs.SouthPolarStereo(-60,-30))


ax.add_feature(cfeature.LAND, facecolor='white', zorder=1)
# ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)


CS = ax.contourf(lon, lat, sst[0, :], 10, cmap='bwr', transform=ccrs.PlateCarree(), zorder=0)
cbar = fig.colorbar(CS, orientation='vertical', label='Temperatura (°C)', shrink=0.7)

# agrego el contorno entre valores y su valor
cs = plt.contour(lon, lat, sst[0, :, :], 5, colors='k', linestyles="dashed", alpha=1, transform=ccrs.PlateCarree())
plt.clabel(cs, inline=1, fmt='%1.1f')

ax.add_feature(cfeature.LAND,facecolor='silver')
ax.add_feature(cfeature.BORDERS, linestyle=':')

plt.title('Temperatura superficial del Mar', loc='center', fontsize=15)
g = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,  zorder=0)
g.ylabels_right = False
g.xlabels_top = False
g.yformatter = LATITUDE_FORMATTER
g.xformatter = LONGITUDE_FORMATTER
g.xlabel_style={'size':12}
g.ylabel_style={'size':12}

### **b. Gráfico de SST en función de la latitud para una dada longitud**



In [ ]:
randlon =  np.random.randint(0, lon.shape[0])

fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(top=10, bottom=9.5)
ax = fig.add_subplot(111)

ax.plot(lat, sst[0, :, randlon], c="red", label="SST", alpha=0.8)
ax.fill_between(lat, 
                sst[0, :, randlon] - err[0, :, randlon], 
                sst[0, :, randlon] + err[0, :, randlon], 
                color="pink", 
                alpha=0.2,
                label="error")

plt.grid()
plt.title(f"SST VS LAT in LON = {lon[randlon]}" )
plt.xlabel("Latitud")
plt.ylabel("Temperatura(°C)")

plt.legend()
plt.show()

### **c. Gráfico de SST promedio (en longitud) para una dada latitud**


In [ ]:
sst_mean = []
sst_std = []
for i, l in enumerate(lon):
    s = sst[0,  :, i]
    sst_mean.append(s.mean())
    sst_std.append(s.std())

df = pd.DataFrame({"lon": lon,  "sst_mean": sst_mean, "sst_std":  sst_std})

In [ ]:
fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(top=10, bottom=9.5)
ax = fig.add_subplot(111)

ax.plot(df.lon, df.sst_mean, label="SST mean")
ax.fill_between(df.lon, 
                df.sst_mean - df.sst_std, 
                df.sst_mean +  df.sst_std, 
                color="c", alpha=.2,
                label="err")

ax.grid(visible=True)
ax.legend()

plt.title(f"SST mean VS LAT", )
plt.xlabel("Latitud")
plt.ylabel("Temperatura (°C)")
plt.show()

## ARCHIVO: SST_noaa_OI_2014_2017.nc

In [ ]:
nf = "SST_noaa_OI_2014_2017.nc"
ds = nc.Dataset(os.path.join(DATADIR, nf))

In [ ]:
lon = ds.variables["lon"][:] - 360
lat = ds.variables["lat"][:]
time =  ds.variables["time"][:]
sst = ds.variables["sst"][:]

# paso el tiempo a datetime 
unit = ds.variables["time"].units.replace("days since ", "")
unit_format = "%Y-%m-%d %H:%M:%S"
dtime = copernicus_to_datetime(time, unit=unit, unit_format=unit_format)

# paso los datetime a str (para tickslabels)
delta = dt.timedelta(days=150)
ticks = to_ticks(dtime[0],  delta, 10)
labelticks = [date_to_str(t, "%Y-%m-%d") for t in ticks]

### **a. Grafique la SST en función del tiempo (Hovmöller) para una dada longitud y para una dada latitud.**


In [ ]:
randlon =  np.random.randint(0, lon.shape[0])
_lon = lon[randlon]
_sst = sst[:, :, randlon]
_time = dtime

plt.figure(figsize=(15, 5), dpi= 60)
plt.pcolormesh(_time, lat, _sst.T, cmap='jet', vmin=5, vmax=25)
plt.xticks(ticks, labelticks, rotation=60, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Latitude', fontsize=20)
# plt.xlabel('Time', fontsize=20)
plt.title(f"Hovmoller SST at LON= {_lon}", fontsize=20)
plt.grid()
plt.show()

### **b. Grafique una serie temporal de SST en un punto del océano**

In [ ]:
randlon =  np.random.randint(0, lon.shape[0]  + 1)
randlat =  np.random.randint(0, lat.shape[0] + 1)
_lon = lon[randlon]
_lat = lat[randlat]
_sst = sst[:, randlat, randlon]
_time = dtime

plt.figure(figsize=(15, 5), dpi= 60)
plt.plot(_time, _sst, "--", c="pink")

plt.xticks(ticks, labelticks, rotation=60, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Latitude', fontsize=20)
plt.ylabel('SST', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.title(f"Serie SST en coordenadas=({_lat}, {_lon})", fontsize=20)
plt.grid()
plt.show()

### **c. Grafique una serie temporal de SST para diferentes latitudes y promedio las longitudes.**

In [ ]:
N = 4
_time = dtime

plt.figure(figsize=(15, 5), dpi= 60)

for n in range(N):
    randlat =  np.random.randint(0, lat.shape[0] + 1)
    _lat = lat[randlat]
    _sst = outofrange_to_nan(sst[:, randlat, :].data, minvalue=-100, maxvalue=100)
    _err = np.nanstd(_sst, axis=1)
    _sst = np.nanmean(_sst, axis=1)
    
    plt.plot(_time, _sst, "--", label=f"LATITUDE: {_lat}")
    plt.fill_between(_time, _sst - _err, _sst + _err,  alpha=.2)


plt.xticks(ticks, labelticks, rotation=60, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Latitude', fontsize=20)
plt.ylabel('SST', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.title(f"Series de SST ", fontsize=20)
plt.grid()
plt.legend()
plt.show()


### **d. Grabe un video de los mapas de SST para el año 2014 (.MOV o .AVI).**

In [ ]:
import matplotlib.axes as maxes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage


def default_map():
    ax = plt.axes(projection=ccrs.PlateCarree())
    # map features
    ax.add_feature(cfeature.LAND, facecolor='white', zorder=1)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    # grid
    g = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,  zorder=0)
    g.ylabels_right = False
    g.xlabels_top = False
    g.yformatter = LATITUDE_FORMATTER
    g.xformatter = LONGITUDE_FORMATTER
    g.xlabel_style={'size':10}
    g.ylabel_style={'size':10}
    
    return ax

In [ ]:
it = 0
iy = 0
year = 2014
inyear = False
while not inyear:
    while (year==dtime[it + iy].year):
        iy+=1
        inyear = True
    it+=1
    
print(f"Dias con datos del {year}: {iy}")

In [ ]:
location="right"
size="5%"
pad="2%"
duration = 60
delta = round(iy/duration)

fig = plt.figure(figsize=(8, 6))
ax = default_map()
divider = make_axes_locatable(ax)
cax = divider.append_axes(location, size=size, pad=size, axes_class=maxes.Axes)

def make_frame(t):
    index = int(delta * np.round(t))
    contourf = ax.contourf(lon, lat, sst[index, :, :], cmap="jet", zorder=0)
    fig.colorbar(contourf,  cax=cax)
    ax.set_title(f"Temperatura superficial del mar\n{date_to_str(dtime[index], '%Y-%m-%d')}")
    g = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,  zorder=0)
    g.ylabels_right = False
    g.xlabels_top = False
    g.yformatter = LATITUDE_FORMATTER
    g.xformatter = LONGITUDE_FORMATTER
    g.xlabel_style={'size':10}
    g.ylabel_style={'size':10}
    return mplfig_to_npimage(fig)

animation = VideoClip(make_frame, duration=duration)

In [ ]:
#animation.ipython_display(fps=1, loop=False, autoplay=False, logger=None)
animation.write_videofile(os.path.join("./images", "TP1.mp4"), fps=1, logger=None)